In [1]:
import os
import datetime
import pandas as pd
import csv
import json
import time

from retrying import retry
from retrying import RetryError
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser

DEVELOPER_KEY=[]
for x in range(2,8):
    if (x==2):
        DEVELOPER_KEY.append(os.environ.get('API_CODE'))
        DEVELOPER_KEY.append(os.environ.get('API_CODE2'))
    else:
        DEVELOPER_KEY.append(os.environ.get('API_CODE'+str(x)))
                             
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
service= build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY[0])

In [2]:
def is_503_error(exception):
    is503=False
    if (isinstance(exception, HttpError)):
        if (exception.resp.status==503):
            is503=True
    return (is503)

In [3]:
@retry(stop_max_attempt_number=7,retry_on_exception=is_503_error, 
       wrap_exception=True,wait_random_min=1000, wait_random_max=2000)
def video_list(dt, dt2, pageToken, channelId):
    videoList=service.search().list(
        part="id,snippet",
        channelId=channelId,
        maxResults=50,
        order="date",
        type="video",
        publishedAfter=dt,
        publishedBefore=dt2,
        pageToken=pageToken
    ).execute()
    
    return videoList

In [4]:
@retry(stop_max_attempt_number=7,retry_on_exception=is_503_error, 
       wrap_exception=True,wait_random_min=1000, wait_random_max=2000)
def video_data(video_id):
    videoData=service.videos().list(
        part="id,snippet,statistics,contentDetails,topicDetails",
        id=video_id
    ).execute()
    
    return videoData

In [5]:
def videos_by_channel(channelId,fname,fname2,ferror):
    this_quota=0
    for year in range (2007,2018,2):
        pageToken=""
        d=datetime.datetime(year,int(1),1,0,0).isoformat()+'Z'
        if (year<2017):
            d2=datetime.datetime(year+1,int(12),31,0,0).isoformat()+'Z'
        else:
            d2=datetime.datetime(year,int(12),31,0,0).isoformat()+'Z'
        
        #gets information until there is no next page
        while (pageToken!='error'):
            this_quota+=100
            try:
                videoList=video_list(d,d2,pageToken,channelId)
                
            except RetryError as e:
                with open(ferror, 'a') as f:
                    f.write("max unsuccessful attempts reached"+" channelId:"+channelId+'\n')
                    this_quota+=16
                continue
                
            #save list of videos if the list is not empty
            if (videoList.get("items",[])!=[]):
                
                with open(fname,'a')as f:
                    out=json.dumps(videoList)
                    f.write(out + '\n')
                    
                for video in videoList.get("items",[]):
                    this_quota+=9
                    try:
                        videoData=video_data(video["id"]["videoId"])
                    except RetryError as e:
                        with open(ferror, 'a') as f:
                            f.write("max unsuccessful attempts reached"+" id:"+video["id"]["videoId"]+'\n')
                            this_quota+=16
                        continue
                    if (videoData.get("items",[])!=[]):
                        with open(fname2,'a')as f2:
                            out2=json.dumps(videoData)
                            f2.write(out2 + '\n')

            try:
                pageToken=videoList['nextPageToken']
            except KeyError as e:
                pageToken='error'

        time.sleep(1)  
        
    return this_quota

In [ ]:
#fileNames=['../minecraft_ch_data_clean.json','../overwatch_ch_data_clean.json','../agario_data_clean.json']
# fileNames=['../LOL_data_clean.json','../callOD_data_clean','../fivenaf_data_clean', '../pkgo_data_clean' ]
# fileNames=['../gta_data_clean.json']
# current 'roblox_data_clean' 'happyW_data_clean.json'

data_folder_channels='/media/aruiz/data/channels_clean_data/'
data_folder_videos='/media/aruiz/data/videos_data/'
channel_file=os.path.join(data_folder_channels,'happyW_data_clean.json')
fileName1=os.path.join(data_folder_videos,'general_videos.json')
fileName2=os.path.join(data_folder_videos,'general_videos_data.json')
fError=os.path.join(data_folder_videos,'errors.txt')
fStatus=os.path.join(data_folder_videos,'listvid_status.txt')

i=0
x=0
quota=0
key=0
service= build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY[key])

with open(fStatus, 'w') as f2:
    with open (channel_file,'r')as f:
        for line in f:
            i+=1
            #if (i>=3000):
            json_dict=json.loads(line)
            channelId=json_dict["id"]   
            this_quota=videos_by_channel(channelId,fileName1,fileName2,fError)
            quota+=this_quota
            f2.write('file:'+str(x)+'\n')
            f2.write('line:'+str(i)+'\n')
            if (quota>=920000):
                quota=0
                key+=1
                if (key==7):
                    key=0                        
                service= build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY[key])
                continue